# Part 3

### Question 3: Parameter-Sharing Scheme for 2D Linear Equivariant Layer

Code for constructing a 4D weight matrix of equivariant layer $F: \mathbb{R}^{(m, n)} \rightarrow \mathbb{R}^{(m, n)}$.
Afterwards we flatten the weight matrix to a 2D matrix of shape $(m n, m n)$.

In [ ]:
import torch
import matplotlib.pyplot as plt

M = 8
N = 7

W = torch.zeros(M, N, M, N)
A = torch.randn(M, N)

for i1 in range(M):
    for j1 in range(N):
        for i2 in range(M):
            for j2 in range(N):
                if i1 == i2 and j1 == j2:
                    W[i1, j1, i2, j2] = 1
                elif i1 == i2 and j1 != j2:
                    W[i1, j1, i2, j2] = 2
                elif i1 != i2 and j1 == j2:
                    W[i1, j1, i2, j2] = 3
                else:
                    W[i1, j1, i2, j2] = 4

plt.imshow(W.reshape(M * N, M * N).to(torch.int32), cmap="tab10")

### Question 5: Parameter-Sharing Scheme for 3D Linear Equivariant Layer

Code for constructing a 6D weight matrix of equivariant layer $F: \mathbb{R}^{(m, n, k)} \rightarrow \mathbb{R}^{(m, n, k)}$.
Afterwards, we flatten the weight matrix to a 2D matrix of shape $(m n k, m n k)$.

In [ ]:
import torch
import matplotlib.pyplot as plt

M = 4
N = 3
K = 5

W = torch.zeros(M, N, K, M, N, K)
A = torch.randn(M, N, K)

for i1 in range(M):
    for j1 in range(N):
        for p1 in range(K):
            for i2 in range(M):
                for j2 in range(N):
                    for p2 in range(K):
                        if i1 == i2 and j1 == j2 and p1 == p2:
                            W[i1, j1, p1, i2, j2, p2] = 1
                        if i1 == i2 and j1 == j2 and p1 != p2:
                            W[i1, j1, p1, i2, j2, p2] = 2
                        if i1 == i2 and j1 != j2 and p1 == p2:
                            W[i1, j1, p1, i2, j2, p2] = 3
                        if i1 == i2 and j1 != j2 and p1 != p2:
                            W[i1, j1, p1, i2, j2, p2] = 4
                        if i1 != i2 and j1 == j2 and p1 == p2:
                            W[i1, j1, p1, i2, j2, p2] = 5
                        if i1 != i2 and j1 == j2 and p1 != p2:
                            W[i1, j1, p1, i2, j2, p2] = 6
                        if i1 != i2 and j1 != j2 and p1 == p2:
                            W[i1, j1, p1, i2, j2, p2] = 7
                        if i1 != i2 and j1 != j2 and p1 != p2:
                            W[i1, j1, p1, i2, j2, p2] = 0

plt.imshow(W.reshape(M * N * K, M * N * K).to(torch.int32), cmap="tab10")

# Part 4

### Question 1: Equivariant Networks

*   *Canonization-based network* - implemented in `src/models.py` file as `CanonicalModel` class.

*   *Symmetrization network and Sampled symmetrization network* - implemented in `src/models.py` file as `SymmetryModel` class. 
    The `SymmetryModel` accepts a permutation list in the constructor for which the input is permuted. If we want to construct full symmetrization network, we can pass all possible permutations to the constructor, while if we want the model to be sampled symmetrization network, we simply pass a (partial) list of some random permutations.

    The permutation list can be generated using helper functions and classes located in `src/permutations.py` file.

*   *Linear Equivariant (and invariant) Layers* - implemented in the `src/layers.py` file as `LinearEquivariant` and `LinearInvariant` classes. 

*   *Data Augmentation* - The augmentation layer is implemented in the `src/permutation.py` file as `RandomPermute` class. This augmentation applies a random permutation to the input data (only during training). 

The full network structures are located in the `experiment.py` file, and are created using `create_mlp_model`, `create_transformer_model` or `create_invariant_model` functions.

A very useful function to create all the networks necessary for the experiments is also located in `experiment.py` file as `get_models` function. Currently, we're not returning the full symmetrization networks from this function, simply uncomment the corresponding lines in the function to return them.


### Question 3: Test Functions

We implemented test functions that test the models for invariance (and equivariance).
The functions `test_invariant` and `test_equivariant` are located in `src/models.py` file. 
We test the equivariance of all relevant models in function `run_invariance_tests` located in the the `experiment.py` file.

# Part 5

It's important to mention that in `main.ipynb` file we perform the following tasks:

* Tets all models and layers for invariance and equivariance.
* Measure inference and forward-backward pass time for all models.
* Train all models on on the required tasks and log results using tensorboard.

The following models were analyzed throughout this experiment:

* *canonical-mlp* - MLP architecture with canonized-inputs        
* *canonical-attn* - Transformer encoder with canonized-inputs       
* *symmetry-mlp* - MLP architecture applied to all input permutations         
* *symmetry-attn* - Transformer encoder applied to all input permutations        
* *symmetry-sampling-mlp* - MLP architecture applied to 10 random input permutations
* *symmetry-sampling-attn* - Transformer encoder applied to 10 random input permutations 
* *intrinsic* - Intrinsic invariant architecture made from equivariant and invariant linear layers            
* *augmented-mlp* - MLP architecture with random permutation applied to the inputs during training       
* *augmented-attn* - Transformer encoder with random permutation applied to the inputs during training       

It's important to note that the symmetry model (without sub-sampling) are not present in the plots since the number of different permutations for $N=10$ is $N!=3628800$, and the model is not able to perform even a single forward pass for this number of permutations. Even when sub-sampling 5% of the permutations, the number of permutations is still too high for the model to handle. As result, in the symmetry-sampling models we sampled 10 random permutations instead.

### Question 1: Plots

The training process was performed until one of the following conditions met:
* Time limit of 30 minutes was surpassed
* The model accuracy didn't improve on the test set for 200 epochs

##### Train-Set 100 ; N = 10

![text](images/train100_seq10_train.png)

![text](images/train100_seq10_test.png)

##### Train-Set 100 ; N = 100

![text](images/train100_seq100_train.png)

![text](images/train100_seq100_train.png)

##### Train-Set 1000 ; N = 10

![text](images/train1000_seq10_train.png)

![text](images/train1000_seq10_test.png)

##### Train-Set 1000 ; N = 100

![text](images/train1000_seq100_train.png)

![text](images/train1000_seq100_train.png)

##### Train-Set 10000 ; N = 10

![text](images/train10000_seq10_train.png)

![text](images/train10000_seq10_test.png)

##### Train-Set 10000 ; N = 100

![text](images/train10000_seq100_train.png)

![text](images/train10000_seq100_train.png)

### Question 2: Tables

#### Invariance:

The following table describes which models are invariant to the input permutations.  
There exist a function to test invariance of models, and indeed its result is consistent with the table.

|          Model         | Invariant |
|------------------------|:---------:|
| canonical-mlp          | True      |
| canonical-attn         | True      |
| symmetry-mlp           | True      |
| symmetry-attn          | True      |
| symmetry-sampling-mlp  | False     |
| symmetry-sampling-attn | False     |
| intrinsic              | True      |
| augmented-mlp          | False     |
| augmented-attn         | False     |

**Notes:**
* *The symmetry model invariance was not tested using the invariance-test function, but this model is inherently invariant from its structure.*

#### Computational Efficiency:   

The following table describes the time it takes for the models to perform inference and training on a single batch.
This table measures the computational efficiency of the models. 

| Model                  | Inference (ms)     | Training (ms)     | Inference (ms)     | Training (ms)     |
| ---------------------- | --------------     | -------------     | --------------     | -------------     |
|                        | **N = 10**         | **N = 10**        | **N = 10**         | **N = 10**        |
| canonical-mlp          | 0.12               | 0.36              | 0.28               | 0.54              |
| canonical-attn         | 0.86               | 4.47              | 10.77              | 46.38             |
| symmetry-sampling-mlp  | 0.6                | 0.95              | 0.99               | 1.56              |
| symmetry-sampling-attn | 11.21              | 46.38             | 110.78             | 482.51            |
| intrinsic              | 0.27               | 0.94              | 1.01               | 2.36              |
| augmented-mlp          | 0.08               | 0.44              | 0.12               | 0.54              |
| augmented-attn         | 0.85               | 4.68              | 11.46              | 48.58             |

**Notes:**
* *We don't include the symmetry models in the table since they are not able to perform the inference at all even when N=10.*
* *The times were measured on input batch of size (32, N, 5) and on a CPU.*

#### Training Time:

To compare the training time between the models, we will measure the time until the training accuracy plateaus.  

| Model                  | DS=100   | DS=1000  | DS=10000  | DS=100    | DS=1000   | DS=10000  |
| ---------------------- | -------- | -------- | --------- | --------- | --------- | --------- |
|                        | **N=10** | **N=10** | **N=10**  | **N=100** | **N=100** | **N=100** |
| canonical-mlp          | 40s      | 1m 40s   | 7m 30 s   | 2m 30s    | 3m 20s    | 7m 30s    |
| canonical-attn         | 5m 15s   | 4m 40s   | 9m        | 24m       | 27m 30s   | 30m (OOT) |
| symmetry-sampling-mlp  | 1m 40s   | 1m 30s   | 10m       | 2m 10s    | 2m 30s    | 13m       |
| symmetry-sampling-attn | 13m      | 24m 40s  | 30m (OOT) | 30m (OOT) | 30m (OOT) | 30m (OOT) |
| intrinsic              | NAN      | 1m 35s   | 12m 20s   | NAN       | 2m        | 12m       |
| augmented-mlp          | 1m 15s   | 1m 15s   | 3m 40s    | 1m 35s    | 1m 45s    | 23m       |
| augmented-attn         | 3m 30s   | 7m       | 9m 30s    | 30m (OOT) | 17m 30s   | 21m 30s   |

**Notes:**
* *We don't include the symmetry models in the table since they are not able to perform the inference at all even when N=10.* 
* *Many models were not able to fully converge in the 30 minutes time limit, hence we add the OOT (Out of Time) label.*  
* *For the intrinsic model, it was not able to train at all for training set size of 100. Therefore put the NAN label there.*


### Question 3: Comparison

Let's compare the different models.

In terms of accuracy, the best model to use is the intrinsic custom architecture, as long as the dataset size is large enough, and $N$ is large enough. 
In all of the scenarios except the one with training set size 100, the intrinsic model outperforms all other models. We explain why the intrinsic model is especially suitable for this task later. In the scenario when the intrinsic model fails to converge (training set size 100), the recommended models are of symmetry-sampling type. 
We believe that the symmetry-sampling technique makes these models more robust in this scenario, since the final prediction is based on the average of the prediction on 10 random input permutations. This averaging technique helps to reduce the variance ("noise") of the training data, and hence makes it more robust to the small training set size.

If inference time is critical and of top priority, we recommend using the augmented-mlp model, which has the lowest inference time. It's important to mention that the canonical-mlp model usually has significantly higher accuracy, while its inference time is only slightly higher than the augmented-mlp model, hence in most scenarios, the canonical-mlp model is the best choice when inference time is of high priority. The intrinsic model is not far behind the canonical-mlp in terms of inference time, hence if inference time is not very critical, one should consider using the intrinsic model. The transformer based models are not recommended for scenarios where inference time is important, as evident from the tables displayed above.

The models with low inference time coincide with the models with low backward-forward pass time during the training. On the other hand, having a fast single-sample backward pass doesn't necessary reflect the total time it takes to train the model (since number of epochs may vary until reaching convergence). The models with the lowest total training time are the MLP-based models, and the intrinsic model. Not very surprisingly, the transformer-based models have the highest training time, since they are significantly more complex.

In terms of scalability, its evident that the attention (transformer) model accuracy is highly dependent on the data dimensionality $N$. Whenever $N$ is low (i.e. 10) the mlp model perform better, but in scenarios when $N$ is high (i.e. 100), the attention model outperforms the mlp model significantly. For the intrinsic-model, the dataset size plays a very significant role in its success. When the dataset size is very small (100), the intrinsic model fails to learn completely, but as long as the dataset size is large enough, the intrinsic model learns the data well. 

Regarding ease of implementation, the model that is the most difficult to implement is the intrinsic model, since it requires custom invariant and equivariant layers. 
The next model in line would probably be the canonization based model. These models use standard layers, but each data sample is first canonized before being fed to the model.
In many scenarios it's not evident what is the canonical form of the data, and how to canonize it (in this task we need to canonize a set of vectors, which is not easy or immediately evident how). The symmetry-based models are easier to implement, since the input data is simply permuted before being fed to the model, but implementing these models efficiently is also not trivial. Efficiency of implementation of the symmetry-based models is crucial since unlike the rest of the models, which feed a single copy of the input to the "internal" model, the symmetry models feed multiple (permuted) copies of the input, hence the inference and training time can be significantly higher.
By far, the easiest models to implement are the augmented models, which use standard layers, and only apply data augmentations to the input data during training. 


### Question 4: Challenges encountered during Implementation:

##### Numeric Errors:

Originally, we constructed the invariant network as follows:

```python

def create_invariant_model(n: int, d: int) -> nn.Module:
    return nn.Sequential(
        LinearEquivariant(in_channels=d, out_channels=10),
        nn.ReLU(),
        LinearEquivariant(in_channels=10, out_channels=10),
        nn.ReLU(),
        LinearInvariant(in_channels=10, out_channels=1),
        nn.Sigmoid(),
    )

```

The problem that arises from this architecture is that the output range of the LinearInvariant is quite large, very often producing large numbers (i.e. 1000).
As result, the sigmoid activation function would saturate, and return binary results (0 or 1) for all inputs. Since the gradient of Sigmoid is zero for outputs 0 or 1, the model would not learn. 

To mitigate this issue I employed two techniques:
* First, in the Custom layers we implemented proper weight and initialization. That partially solved the problem, but not entirely.

* Second, I added a `nn.BatchNorm1d(num_features=1)` layer after the `LinearInvariant` layer, right before the sigmoid function. It learns the mean and variance of the outputS of the `LinearInvariant` layer during the training process, and normalizes it to variance 1 and expectancy 0 (on average). It's important to note that the batch normalization layer keeps the model invariant, since it simply multiplies scalar outputs by a learned scalar factor and adds a learned scalar bias. Indeed, using the batch normalization layer solved the problem of the model not learning. Since the batch normalization layer normalizes the inputs to the sigmoid function, it no longer saturates and the model can learn.
Although it might take a while for the `BatchNorm1d` layer to learn the proper normalization coefficients, once it does the model trains very quickly. 

* I believe another approach for this issue is in the equivariant and invariant layers to average the outputs over the size of `in_channels`, since each output is the sum of `in_channels` different equivariant (or invariant) layers. Averaging still retains the equivariant (or invariant) property of the layer, but helps reduce the output range.

Resulting Architecture:

```python
 def create_invariant_model(n: int, d: int) -> nn.Module:
    return nn.Sequential(
        LinearEquivariant(in_channels=d, out_channels=10),
        nn.ReLU(),
        LinearEquivariant(in_channels=10, out_channels=10),
        nn.ReLU(),
        LinearInvariant(in_channels=10, out_channels=1),
        nn.BatchNorm1d(1),
        nn.Sigmoid(),
    )
```

##### Overfitting:
Another big issue we encountered was overfitting. To overcome it, we added an option to dynamically generate the data every time the `Dataset` is accessed. 
This way, the model never sees the same data twice, and not able to overfit. That indeed resolved completely the overfitting issue.

##### Canonization:
The input data is of $(N, D)$ dimension, with $S_n$ permutation invariance over the rows. Each row is a vector of length $D$. 
In order to define a canonization function, we need to define some inherent ordering between the rows. 
There is no immediate straight-forward way of ordering vectors, hence the challenge.   
The approach that I decided to implement is ordering the vectors by the mean of their elements + the first element of the vector. i.e. with each vector $\vec{x}$ we associate the value $f(\vec{x}) = \text{mean}(\vec{x}) + \vec{x}_0$.   
This approach is not perfect (there are ways to "cheat" it), but there are ways to achieve true canonization, but these approaches are way more computationally intense. 

Example for true canonization function:
1) Given a set of vectors, first order them according to the first element of the vector.
2) Then, order them according to the second element of the vector, but only within the groups of vectors that have the same first element.
3) Continue this process until all elements are ordered or until the maximum number of elements is reached.

##### Symmetrization Network:
The symmetrization network is a very powerful tool to learn equivariant functions. However, it is computationally expensive
and tricky to implement efficiently. Our implementation balances performance and memory utilization by forwarding through the network multiple 
permuted versions of the input data at once (by creating a super-batch). We can control that number of forwarded permutations to balance performance and memory (more permutations - better performance but higher memory utilization).

### Question 5: Recommendations

There are several different scenarios that we would like to address:

* If the training set is large enough such that the intrinsic model manages to converge, we recommend using it. That model exhibits the best performance in terms of accuracy by a significant margin, is efficient in terms of inference and training time, and is invariant to the input permutations. Hence, we recommend using the intrinsic model whenever possible.

* Now we discuss the scenarios when when the training set is very small such that the intrinsic model fails to converge.
    * If training and inference resources (computational resources) are limited, then one shouldn't use the transformer (attn) models, nor the symmetry-sampling ones, since they are significantly more computationally expensive. Hence, the choice becomes between canonical-mlp and augmented-mlp models. In all relevant cases the canonical-mlp has better accuracy by a quite significant margin, hence we recommend using it.
    
    * If strong guarantees on invariance are not required (i.e. models are semi-invariant, but not mathematically guarantee invariance in all cases), then the symmetry-sampling models are recommended in terms of accuracy.
        * For low $N$ values, the symmetry-sampling-mlp model is recommended
        * On the other hand, for high $N$ values, the symmetry-sampling-attn model is recommended.
    * If invariance is required, then the canonical models are recommended. These models guarantee invariance, while also exhibiting good accuracy.
        * For low $N$ values, the canonical-mlp model is recommended
        * For high $N$ values, the canonical-attn model is recommended.

*Note: We practically never recommend using the regular symmetrization models, since even for very small $n$ values their training and inference time becomes astronomical (at least for $S_n$ invariance). Similarly, the augmented model is not recommended since it has significantly lower accuracy than other models, which are competitive with it in terms of inference time.*

<p align="center">
  <img src="images/model-recommendations.png" />
</p>

### Question 6 and 7:

The reason that the equivariant architecture is especially suited for this task is the following:

Given $d$ dimensional random vector $x = (x_1, x_2, ..., x_d) \sim \mathcal{D^d}$, the formula for the empirical variance is as follows:

$$ {Var}_{em}(x) = \frac{1}{d - 1} \sum_{i=1}^{d} (x_i - \bar{x})^2 $$

$$ \bar{x} = \frac{1}{d} \sum_{i=1}^{d} x_i $$

Notice that this calculation can be easily achieved by the the following invariant architecture:

($I_{d \times d}$ denotes the identity matrix, $1_{d \times d}$ denotes the matrix filled with ones, and $0_{d}$ denotes the zero vector)

Let $N = \phi \circ \alpha \circ F$ be an invariant network, where $\phi$ is invariant layer, $\alpha$ is pointwise activation, and $F$ is an equivariant layer.

* Define $F : \mathbb{R}^d \rightarrow \mathbb{R}^d$ as follows: $ F = 1 \cdot I_{d \times d} + \frac{-1}{d} \cdot {1}_{d \times d} + 0_{d}$
(recall that general equivariant $F$ is of the form $ F = \alpha \cdot I_{d \times d} + \beta \cdot {1}_{d \times d} + b$)

* Define $\alpha : \mathbb{R} \rightarrow \mathbb{R}$ as follows: $ \alpha(x) = x^2 $

* Define $\phi : \mathbb{R}^d \rightarrow \mathbb{R}$ as follows: $ \phi = \frac{1}{d-1} \cdot 1_{d} + 0$ (recall that general invariant $\phi$ is of the form $ \phi = \alpha \cdot 1_{d} + b$)

As result, we get:

$$ F(x)_i = x_i - \frac{1}{d} \sum_{j=1}^{d} x_j + 0 = x_i - \bar{x} $$

$$ \alpha(F(x))_i = F(x)_i^2 = (x_i - \bar{x})^2 $$

$$ N(x) = \frac{1}{d-1} \sum_{i=1}^{d} \alpha(F(x))_i = \frac{1}{d-1} \sum_{i=1}^{d} (x_i - \bar{x})^2 = \text{Var}_{em}(x) $$

We showed that using the invariant and equivariant layers only, we were able to calculate the empirical variance of an input sample. 
Since the task of the model is to differentiate between inputs generated from distributions with different variances, the equivariant architecture is especially suited for this task.
Notice the small number of parameters required to calculate the variance (3 parameters in layer $F$ and two parameters in layer $\phi$), hence the optimization process of this architecture is easier than for other architectures.

*Note: We showed how to calculate the variance across a single feature dimension, but the same idea holds for calculating the variance for each feature for element of size $(n \times d)$, and afterwards averaging the empirical variances across the feature dimension to get the final variance of the sample.*

### Question 8:

Currently, we're using the symmetry group $S_n$ over the channel dimensions.
A better symmetry group to use would be $S_n \times S_d$ when $S_n$ acts on the channel dimension and $S_d$ acts on the feature dimension. The reason this symmetry group is suitable is because each feature is i.i.d. vector of length $d$ generated from a normal distribution, and any permutation of the vector does not change the probability of it being generated from the underlying distribution that generated it. Since the model tries to detect the underlying distribution, it should be invariant to permutations of the feature dimensions.

Formally:

$$ \Pr(x \sim \mathcal{N}(0, I) \; | \; x) = 
\Pr(\sigma \cdot x \sim \mathcal{N}(0, I) \; | \; x ) \; \forall \sigma \in S_d \Rightarrow $$

$$ \Pr(x_1, x_2, ... x_n \sim \mathcal{N}(0, I) \; | \; x_1, x_2, ... x_n) = 
\Pr(\sigma \cdot x_1, \sigma \cdot x_2, ... \sigma \cdot x_n \sim \mathcal{N}(0, I) \; | \; x_1, x_2, ... ,x_n) \; \forall \sigma \in S_d  $$

when $x_i$ is a feature vector of length $d$ and $\sigma$ is a permutation of the feature dimensions
(remember that each input sample is composed of $n$ feature vectors of length $d$).